Teams:
- JO: jong ondernemen (thematische opleidingen, trajecten, etc.)
- DO: duurzaam ondernemen (thematische opleidingen, trajecten, etc.)
- IN: innovatie en digitalisering (thematische opleidingen, trajecten, etc.)
- IO: internationaal ondernemen (thematische opleidingen, trajecten, etc.)
- AO: arbeidsmarkt en onderwijs (thematische opleidingen, trajecten, etc.)
- EX: expert (thematische opleidingen, trajecten, etc.)
- GR: Groei: biedt vooral **Plato (P)** en accelero aan
- P: Plato
- BB: Belangenbehartiging
- CO: communicatie (verkoopt publiciteit, voert publiciteitscampagnes uit, etc.)
- NW: netwerking (organiseert evenementen)
- HA:  Haven (thematische opleidingen, trajecten, etc.)
- MA: match (verkoopt lidmaatschappen)

In [1]:
import pandas as pd 
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, MetaData, select


load_dotenv()
csv_dir =  '/home/flor/Workspace/DEP2/DataEngineerProj2/Data'
SERVER = os.environ.get('SERVER')
DATABASE = os.environ.get('DATAWAREHOUSE')
UID = os.environ.get('USER')
PWD = os.environ.get('PASSWORD')

connection_string = f'mssql+pyodbc://{UID}:{PWD}@{SERVER}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

metadata = MetaData()
metadata.reflect(engine)

In [2]:
df_fact = pd.read_sql_table(table_name='FactCampagne', con=engine)

Namen = df_fact['Naam_Campagne'].tolist()
split = []

for naam in Namen:
    split_naam = naam.split('-')
    split.append(split_naam)

onderwerpen = []

for item in split:
    if len(item) > 1:
        onderwerpen.append(item[1].upper())
    else:
        onderwerpen.append(None)  # or some default value if you prefer

df_fact['Onderwerp_campagne'] = onderwerpen

onderwerp_mapping = {
    'JO': 'Jong Ondernemen',
    'DO': 'Duurzaam Ondernemen',
    'IN': 'Innovatie en Digitalisering',
    'IO': 'Internationaal Ondernemen',
    'AO': 'Arbeidsmarkt en Onderwijs',
    'EX': 'Expert',
    'GR': 'Groei',
    'P': 'Plato',
    'BB': 'Belangenbehartiging',
    'CO': 'Communicatie',
    'NW': 'Netwerking',
    'HA': 'Haven',
    'MA': 'MATCH',
}

# Maak een kopie van de oorspronkelijke kolom om de NaN-waarden te vervangen
df_fact['Onderwerp_Oorspronkelijk'] = df_fact['Onderwerp_campagne']

# Map naar afkortingen
afkorting_mapping = {v: k for k, v in onderwerp_mapping.items()}
df_fact['Onderwerp_campagne'] = df_fact['Onderwerp_campagne'].map(afkorting_mapping)

# Map naar de volledige beschrijving en vervang NaN door de originele waarde
df_fact['Onderwerp_campagne'] = df_fact['Onderwerp_campagne'].map(onderwerp_mapping).fillna(df_fact['Onderwerp_Oorspronkelijk'])

# Drop de tijdelijke kolom
df_fact = df_fact.drop('Onderwerp_Oorspronkelijk', axis=1)

df_fact.to_sql('FactCampagne', con=engine, schema='dbo', if_exists='replace', index=False)


93